In [1]:
import torch
import matplotlib.pyplot as plt
import mlx.core as mx

In [ ]:
n=2000
alpha = torch.tensor([1/(i+1) for i in range(n)])
a = torch.tril(torch.ones(n,n))
b = torch.randint(10, (n,1)).float()
# a, b = mx.array(a), mx.array(b)
c = (a*alpha)@ b
#sample
d = c[::100]
plt.plot(c)
print(b.mean())

: 